In [1]:
import pandas as pd
import numpy as np
import eli5

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from math import pi
import heapq

import matplotlib.pyplot as plt
from pathlib import Path

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

Using TensorFlow backend.


In [2]:
file="data/train.json"
data = pd.read_json(file, orient='columns')
data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
ingredients = data['ingredients']
all_ingredients = []
for item in ingredients:
    all_ingredients.extend(item)
    
#This list will need some cleanup. whole lotta cleanup for data visualization
unique_ingredients = list(set(all_ingredients))
# unique_ingredients

print("all_ingredients: " + str(len(all_ingredients)))
print("unique_ingredients: " + str(len(unique_ingredients)))

all_ingredients: 428275
unique_ingredients: 6714



# machine learning

In [7]:
cuisine_list = data['cuisine']
cuisine_compilation = []
for cuisine in cuisine_list:
    cuisine_compilation.append(cuisine)
      
cuis_unique = list(set(cuisine_compilation))

label_encoder_cuis = LabelEncoder()
label_encoder_cuis.fit(cuis_unique)
encoded_cuis = label_encoder_cuis.transform(data['cuisine'])

one_hot_cuis = to_categorical(encoded_cuis)
one_hot_cuis[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [8]:
data["one_hot_cuisine"] = list(one_hot_cuis)
data = data[["cuisine", "one_hot_cuisine", "ingredients"]]
data.head()

,cuisine,one_hot_cuisine,ingredients
0,greek,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[water, vegetable oil, wheat, salt]"
4,indian,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[black pepper, shallots, cornflour, cayenne pe..."


In [9]:
ingredients = data.loc[:,'ingredients']

i_map = {}
i_list = []
counter = 0
for lists in ingredients:
    for items in lists:
        if items not in i_map:
            i_list.append(items)
            i_map[items] = counter
            counter = counter + 1

ingredients_encodings = []
for lists in ingredients:
    encoding = [0]*len(i_map)
    for items in lists:
        encoding[i_map[items]] = 1
    ingredients_encodings.append(encoding)

In [10]:
data["one_hot_ingredients"] = ingredients_encodings
one_hot_cuisine = data['one_hot_cuisine']
one_hot_ingredients = data['one_hot_ingredients']

In [11]:
ing_train, ing_test,cuis_train, cuis_test = train_test_split(one_hot_ingredients, one_hot_cuisine, random_state=1)

cuis_train = np.array(cuis_train.tolist())
cuis_test = np.array(cuis_test.tolist())
ing_train = np.array(ing_train.tolist())
ing_test = np.array(ing_test.tolist())

model = Sequential()
number_inputs = len(i_list)
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#unique ingredients
number_classes = 6714
model.add(Dense(units=number_classes, activation='softmax'))

#unique cuisines
number_classes = 20
model.add(Dense(units=number_classes, activation='softmax'))

In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 26860     
_________________________________________________________________
dense_1 (Dense)              (None, 6714)              33570     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                134300    
Total params: 194,730
Trainable params: 194,730
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    ing_train,
    cuis_train,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
 - 11s - loss: 0.9773 - acc: 0.7017
Epoch 2/20
 - 13s - loss: 0.9509 - acc: 0.7121
Epoch 3/20
 - 13s - loss: 0.9233 - acc: 0.7245
Epoch 4/20
 - 14s - loss: 0.8996 - acc: 0.7307
Epoch 5/20
 - 14s - loss: 0.8787 - acc: 0.7366
Epoch 6/20
 - 14s - loss: 0.8608 - acc: 0.7449
Epoch 7/20
 - 14s - loss: 0.8446 - acc: 0.7482
Epoch 8/20
 - 14s - loss: 0.8299 - acc: 0.7529
Epoch 9/20
 - 14s - loss: 0.8146 - acc: 0.7594
Epoch 10/20
 - 15s - loss: 0.8020 - acc: 0.7612
Epoch 11/20
 - 15s - loss: 0.7892 - acc: 0.7645
Epoch 12/20
 - 13s - loss: 0.7775 - acc: 0.7681
Epoch 13/20
 - 10s - loss: 0.7666 - acc: 0.7753
Epoch 14/20
 - 10s - loss: 0.7557 - acc: 0.7800
Epoch 15/20
 - 11s - loss: 0.7453 - acc: 0.7838
Epoch 16/20
 - 11s - loss: 0.7368 - acc: 0.7857
Epoch 17/20
 - 10s - loss: 0.7280 - acc: 0.7895
Epoch 18/20
 - 10s - loss: 0.7200 - acc: 0.7917
Epoch 19/20
 - 10s - loss: 0.7122 - acc: 0.7936
Epoch 20/20
 - 11s - loss: 0.7056 - acc: 0.7981
